### prepare SSAGA for the NDA - download data (sans restricted data) from PreRelease folder

In [ ]:
import os, datetime
import csv
import sys
import shutil
from openpyxl import load_workbook
import pandas as pd
from io import BytesIO
import numpy as np
import subprocess
from scipy import stats
from ccf.box import LifespanBox
from ccf.config import LoadSettings
from ccf.redcap import RedcapTable 
config = LoadSettings()
eventlist=['V1','V2']

In [ ]:
box=LifespanBox(cache='./')
verbose = True
snapshotdate = datetime.datetime.today().strftime('%m_%d_%Y')
pathout="./prepped/hca" 

#Rosetta (a.k.a Inventory) file will have all the nda vars and pedids
extrainfo=config['rosetta']['filename']

redcapconfigfile="/home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/.boxApp/redcapconfig.csv"

snapshotdate = datetime.datetime.today().strftime('%m_%d_%Y')
#catfromdate=max of last run--'2019-06-17'
snapshotdate

In [ ]:
inventory=pd.read_csv(extrainfo)[['subject','pseudo_guid','nda_interview_date','nda_age','M/F','pedid','redcap_event_name','redcap_event','Curated_SSAGA']]
inventory=inventory.loc[inventory.nda_age.isnull()==False]
inventory.nda_interview_date=pd.to_datetime(inventory.nda_interview_date).dt.strftime('%m/%d/%Y')
inventory.nda_age=inventory.nda_age.round(0).astype(int)
print('Inventory Shape out of the Box',inventory.shape)
print('Number of Unique Subjects',len(inventory.subject.unique()))
inventory=inventory.loc[inventory.Curated_SSAGA.isin(['YES','YES BUT'])]#1789 on 11/19/21
inventory=inventory.loc[inventory.redcap_event.isin(eventlist)]
inventory.shape


In [ ]:
handle=[926768966179]
ssagad=box.download_files(handle)

In [ ]:
datafile='HCA_SSAGA_2022-03-04.csv'
ssagadata=pd.read_csv(datafile)
print(ssagadata.shape)
ssagadata=ssagadata.drop(columns=['hcpa_id','flagged','study','register_subject_complete','vist1','vist2','register_visit_1_complete','register_visit_2_complete','redcap_event_name','ssaga_complete'])
ssagadropsubs=pd.merge(ssagadata, inventory, on=['subject','redcap_event'], how='right').drop(columns=['pedid', 'redcap_event_name','Curated_SSAGA'])
ssagadropsubs.shape

In [ ]:
ssagas=ssagadropsubs.rename(columns={'subject':'src_subject_id','M/F':'sex','nda_age':'interview_age','nda_interview_date':'interview_date','pseudo_guid':'subjectkey', 'redcap_event':'event'})


In [ ]:
#import map
mapfile=pd.read_excel('./Mapping Key.xlsx', sheet_name='HCP_SSAGA_IV_15Dec2021')
mapfile['SSAGA Element Name'].head(20)

listnames=list(mapfile['SSAGA Element Name'].str.strip())

for i in listnames:
    if i not in list(ssagas.columns):
        print(i,'is not in map')


In [ ]:
#import options
pd.options.display.max_colwidth=100

print(mapfile.loc[mapfile['SSAGA Element Name']=='dr1'][['Notes']])
##ssagas[['dr1']]#.columns

dr1_list = ['dr1___1','dr1___2','dr1___3','dr1___4','dr1___5','dr1___6','dr1___7','dr1___8','dr1___9']
elements_list = ['dr1_1','dr1_2','dr1_3','dr1_4','dr1_5','dr1_6','dr1_7','dr1_8','dr1_9']
dr1 = ssagas[dr1_list].sum(axis=1).value_counts()
#for COC,STIM,SED,OPIATE,PCP,HAL,SOL,COMB and OTHER

In [ ]:
#split into the two structures
mapfile.columns
mapfile.head()
mapfile['Short Name'].value_counts()

In [ ]:
part1=mapfile.loc[mapfile['Short Name'].str.contains('ssaga_part101')]
part2=mapfile.loc[mapfile['Short Name'].str.contains('ssaga_part201')]

In [ ]:
part1['Short Name'].value_counts()
part2['Short Name'].value_counts()

In [ ]:
part1['Short Name'].head()
part1.head()

In [ ]:
part1.head()
struct101=ssagas[list(part1['SSAGA Element Name'])] #.head()
rename1=dict(zip(list(part1['SSAGA Element Name']),list(part1['NDA Element Name'])))

#remove dr1 and replace with dummy vars.
part2b=part2.loc[~(part2['SSAGA Element Name']=='dr1')]
dr1_list = ['dr1___1','dr1___2','dr1___3','dr1___4','dr1___5','dr1___6','dr1___7','dr1___8','dr1___9']


elements_list = ['dr1_1','dr1_2','dr1_3','dr1_4','dr1_5','dr1_6','dr1_7','dr1_8','dr1_09']
struct201=ssagas[list(part2b['SSAGA Element Name'])+dr1_list]
rename2=dict(zip(list(part2b['SSAGA Element Name'])+dr1_list,list(part2b['NDA Element Name'])+elements_list))


In [ ]:
len(list(part2b['NDA Element Name'])+elements_list)
len(list(part2b['SSAGA Element Name'])+dr1_list)

In [ ]:
ready1=struct101.drop(columns=['event']).rename(columns=rename1)
ready2=struct201.drop(columns=['event']).rename(columns=rename2)

In [ ]:
#Write out structure for validation
filePath=pathout+'/ssaga_part101.csv'
if os.path.exists(filePath):
    os.remove(filePath)
else:
    print("Can not delete the file as it doesn't exists")

with open(filePath,'a') as f:
    f.write("ssaga_part1,1\n")
    ready1.to_csv(f,index=False)

    
#Write out structure for validation
filePath=pathout+'/ssaga_part201.csv'
if os.path.exists(filePath):
    os.remove(filePath)
else:
    print("Can not delete the file as it doesn't exists")

with open(filePath,'a') as f:
    f.write("ssaga_part2,1\n")
    ready2.to_csv(f,index=False)